# Wizeline Data Engieneering Bootcamp, Challenge

## Introduction

The main of this project it is to process a larga dataset provided by the PROFECO (Mexican Consumer Protection Agency) and make an exploratory analysis that would give us a better understanding of the data stored there.

## Report & Code of the Solution
### General Imports

In [10]:
import pandas as pd
import time # Used to meseaure the performance of the code.

### Importing data to Pandas Dataframe.
In my first try, I tried to open the dataset in the usual way to open, unfortunately this was too slow and sometimes throwed some errors due to the size of the file.

* Code
```python
start = time.time()
data = pd.read_csv("data/all_data.csv")
end = time.time()
print("Read csv in the 'traditional' way: ",(end-start),"sec")
```
* Result:
```
/Users/benny/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (5,13,14) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
Read csv in the 'traditional' way:  266.19145798683167 sec
```
* Shape of the data:
```
(62530715, 15)
```

Shape of our data:

In [11]:
data.shape

(62530715, 15)

Using the alternitve method:

In [7]:
start = time.time()
chunk = pd.read_csv('data/all_data.csv',chunksize=1000000)
end = time.time()
data = pd.concat(chunk)
print("Read csv with chunks: ",(end-start),"sec")

Read csv with chunks:  0.002966165542602539 sec


In [22]:
data.shape

(62530715, 15)

### Exploratory Analysis
Now that I imported the data, its time to start wit the exploratory analysis. In my professional experience (that it's not much yet) I discovered that the speed of Pandas depedends directly of the number of columns, so for my solution, I will make a serie of subsets in order to get the results faster.


#### 1. How many commercial chains are monitored, and therefore, included in this database?
Code:

In [50]:
cc = data['cadenaComercial'].copy()
nan_value = float("NaN")
cc.replace("", nan_value, inplace=True) #Deleting empty records
cc = cc.dropna() #I use this panda's function to ensure that I am not counting null values.
monitored = len(cc.unique())

print(f'There are {monitored} commercials chains monitored in this database.')
del cc

There are 705 commercials chains monitored in this database.


#### 2. What are the top 10 monitored products by State?
Code:

In [125]:
top10 = data[['producto','estado']].copy()
# While analyzing if the states list it's ok I found that there is a element called 'estado' that doesn't match with the name of any of the Mexican States, Also I discover a nan record.
# so I decided to quit this rows from my df in order to guarantee a correct analysis
nan_value = float("NaN")
top10.replace("", nan_value, inplace=True) #Deleting empty records
top10 = top10.dropna() #I use this panda's function to ensure that I am not counting null values.
top10 = top10[top10['estado']!= 'estado']
monitored = top10['estado'].unique()
# print(len(monitored)) # It's correct, Mexico has 32 states.
top10 = top10.value_counts().to_frame('n_products').reset_index()
top10 = top10.sort_values(['estado','n_products'], ascending = False)
top10 = top10.groupby('estado').head(10).reset_index()
del top10['index']
display(top10)
del top10

,producto,estado,n_products
0,DETERGENTE P/ROPA,ZACATECAS,20884
1,LECHE ULTRAPASTEURIZADA,ZACATECAS,17309
2,LAVADORAS,ZACATECAS,16072
3,MAYONESA,ZACATECAS,15927
4,JABON DE TOCADOR,ZACATECAS,15926
...,...,...,...
315,DESODORANTE,AGUASCALIENTES,8859
316,JABON DE TOCADOR,AGUASCALIENTES,8517
317,CHILES EN LATA,AGUASCALIENTES,7946
318,YOGHURT,AGUASCALIENTES,7401


#### 3. Which is the commercial chain with the highest number of monitored products?
Code:


In [109]:
cc = data[['cadenaComercial','producto']].copy()
# While analyzing if the states list it's ok I found that there is a element called 'estado' that doesn't match with the name of any of the Mexican States, Also I discover a nan record.
# so I decided to quit this rows from my df in order to guarantee a correct analysis
nan_value = float("NaN")
cc.replace("", nan_value, inplace=True) #Deleting empty records
cc = cc.dropna() #I use this panda's function to ensure that I am not counting null values.
cc = cc.groupby('cadenaComercial')['producto'].count().nlargest(1)
cc

cadenaComercial
WAL-MART    8643133
Name: producto, dtype: int64

#### 4. Use the data to find an interesting fact.

I would like to know the top 5 most expensive and the cheapest product monitored by the PROFECO.

Code:


In [123]:
prices = data[['producto','precio','presentacion']].copy()
# While analyzing if the states list it's ok I found that there is a element called 'estado' that doesn't match with the name of any of the Mexican States, Also I discover a nan record.
# so I decided to quit this rows from my df in order to guarantee a correct analysis
nan_value = float("NaN")
prices.replace("", nan_value, inplace=True) #Deleting empty records
prices = prices.dropna() #I use this panda's function to ensure that I am not counting null values.
prices['precio'] = pd.to_numeric(prices["precio"],errors='coerce') ## Ignoring errors
prices = prices.sort_values(['precio'])
print("Top 5 Cheapest monitored products:")
display(prices.head(5))
print("Top 5 Most Expensive monitored products:")
prices = prices.sort_values(['precio'],ascending=False)
display(prices.head(5))


Top 5 Cheapest monitored products:


,producto,precio,presentacion
55516133,CHILE SECO,0.10,BOLSA 100 GR. PASILLA
16351042,PASTA PARA SOPA,0.10,PAQUETE 200 GR. SPAGHETTI
61433448,LECHUGA,0.10,PZA. ROMANA
14758682,MARGARINA,0.25,BARRA 90 GR. SIN SAL
333754,ACELGA,0.45,MANOJO


Top 5 Most Expensive monitored products:


,producto,precio,presentacion
33061685,PANTALLAS,299999.00,84 LM 9600. 84 PLGS. LED 3D. ENTRADA USB
27799092,PANTALLAS,299999.00,84 LM 9600. 84 PLGS. LED 3D. ENTRADA USB
29812877,PANTALLAS,299999.00,84 LM 9600. 84 PLGS. LED 3D. ENTRADA USB
29607564,PANTALLAS,254999.15,84 LM 9600. 84 PLGS. LED 3D. ENTRADA USB
27851686,PANTALLAS,254999.15,84 LM 9600. 84 PLGS. LED 3D. ENTRADA USB


#### 5. What are the lessons learned from this exercise?
In this excersise I learn how to manipulate large databases, cause until this moment I only have been working with medium size databases, max 400,000 records, So I feel more prepared when a situation like this appear.

#### 6. Can you identify other ways to approach this problem? Explain.
Yes!
At the moment of investigate how to process this large database, I discover an alternative to pandas called Dask that I didn't use cause in the requeriements was mandatory to use pandas, but I could see that this library process the data faster due to his paralell computing process.
Also when I saw the shape of the df the first thing that came to my mind was the word "Spark", but I still in the process of know how it works.